In [38]:
from transformers.pipelines import pipeline, Pipeline
from compiled_tokenizer import CompiledTokenizer
from compiled_model import CompiledModel
import torch
import torch.neuron

In [30]:
sample_input = 'This is an example sentence. We use it to compare tokenizers and their compiled variants.'

In [31]:
hf_pipeline = pipeline('feature-extraction',"sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

In [32]:
type(hf_pipeline)

transformers.pipelines.feature_extraction.FeatureExtractionPipeline

In [33]:
hf_pipeline_output = hf_pipeline(sample_input)

In [34]:
type(hf_pipeline_output), len(hf_pipeline_output), len(hf_pipeline_output[0]), len(hf_pipeline_output[0][0]) # 1 x n_tokens x n_embedd_dim 

(list, 1, 25, 384)

In [35]:
hf_pipeline_output[0][0][:15]

[0.2016810029745102,
 0.28019487857818604,
 0.1350724846124649,
 -0.5622077584266663,
 -0.02432403899729252,
 -0.23179000616073608,
 0.4285317063331604,
 0.4974362552165985,
 -0.13233201205730438,
 -0.2266368865966797,
 -0.2930293083190918,
 -0.14483046531677246,
 -0.1874832659959793,
 -0.5158872604370117,
 0.419760137796402]

In [82]:
compiled_tokenizer = CompiledTokenizer.from_pretrained('./compiled_tokenizer')
compiled_model = CompiledModel.from_pretrained('./compiled_model')
print(type(compiled_tokenizer), type(compiled_model))
hf_compiled_pipeline = pipeline('feature-extraction',tokenizer=compiled_tokenizer, model=compiled_model)
print(type(hf_compiled_pipeline))

<class 'transformers.models.bert.tokenization_bert_fast.BertTokenizerFast'> <class 'compiled_model.CompiledModel'>
<class 'transformers.pipelines.feature_extraction.FeatureExtractionPipeline'>


In [83]:
type(hf_compiled_pipeline)

transformers.pipelines.feature_extraction.FeatureExtractionPipeline

In [85]:
hf_compiled_pipeline.tokenizer.tokenization_settings, hf_compiled_pipeline.tokenizer.compiled

({'padding': 'max_length',
  'truncation': True,
  'add_special_tokens': True,
  'max_length': 50},
 True)

In [87]:
hf_compiled_pipeline_configured_output = hf_compiled_pipeline.tokenizer.configured_tokenize(sample_input)
hf_compiled_pipeline_configured_output

{'input_ids': [0, 3293, 83, 142, 27781, 149357, 5, 1401, 4527, 442, 47, 69101, 47, 1098, 52825, 7, 136, 2363, 375, 5974, 297, 20117, 7, 5, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [88]:
hf_compiled_pipeline_output = hf_compiled_pipeline(sample_input,tokenize_kwargs=hf_compiled_pipeline.tokenizer.tokenization_settings)

Model output is a dictionary. Converting


In [89]:
type(hf_compiled_pipeline_output), len(hf_compiled_pipeline_output[0]), len(hf_compiled_pipeline_output[0][0]) # 1 x n_tokens x n_embedd_dim 

(list, 50, 384)

In [90]:
hf_compiled_pipeline_output[0][0][:15]

[0.20376674830913544,
 0.2765544056892395,
 0.13411857187747955,
 -0.5617780685424805,
 -0.024763502180576324,
 -0.22976504266262054,
 0.4279550313949585,
 0.49747270345687866,
 -0.13185089826583862,
 -0.22668159008026123,
 -0.28989726305007935,
 -0.1457303762435913,
 -0.18916262686252594,
 -0.5175866484642029,
 0.4199066758155823]

In [98]:
token_index = 24# 0-24
embedding_vector_range = (0,300) # 0-382,1-383

for embedding_entry_hf, embedding_entry_compiled in zip(
    hf_pipeline_output[0][token_index][embedding_vector_range[0]:embedding_vector_range[1]], 
    hf_compiled_pipeline_output[0][token_index][embedding_vector_range[0]:embedding_vector_range[1]]
):
    print(f'{embedding_entry_hf} (generic hf) | {embedding_entry_compiled} (compiled)')

0.2015596479177475 (generic hf) | 0.20426766574382782 (compiled)
0.28036993741989136 (generic hf) | 0.2775125205516815 (compiled)
0.13522349298000336 (generic hf) | 0.1341019719839096 (compiled)
-0.5623601675033569 (generic hf) | -0.5606499910354614 (compiled)
-0.024141062051057816 (generic hf) | -0.024146482348442078 (compiled)
-0.23195278644561768 (generic hf) | -0.22975675761699677 (compiled)
0.4286251962184906 (generic hf) | 0.4270547926425934 (compiled)
0.4972536861896515 (generic hf) | 0.4963544011116028 (compiled)
-0.13238519430160522 (generic hf) | -0.13068823516368866 (compiled)
-0.22663092613220215 (generic hf) | -0.22723890841007233 (compiled)
-0.2931181490421295 (generic hf) | -0.2902343273162842 (compiled)
-0.14490890502929688 (generic hf) | -0.14671768248081207 (compiled)
-0.18754956126213074 (generic hf) | -0.18938593566417694 (compiled)
-0.5160385370254517 (generic hf) | -0.5172978043556213 (compiled)
0.4198281466960907 (generic hf) | 0.42024022340774536 (compiled)
-0.7